In [2]:
# !unzip -uq "dataset_5m.zip" -d "dataset_5m/"
# !unzip -uq "dataset_pred.zip" -d "dataset_pred/"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
path_plus='dataset_5m/dataset_5m/E01' # 무표정
path_zero='dataset_5m/dataset_5m/E02' # 웃음
path_minus='dataset_5m/dataset_5m/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:14400
plus 경로에 저장된 파일의 개수:14400
minus 경로에 저장된 파일의 개수:14400


In [3]:
xsize=100 # size 100*100 => 200*200으로 변경
ysize=100

In [4]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, grayscale=False, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [5]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
print(x.shape)

(43200, 100, 100, 3)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

input_shape = (xsize,ysize, 3)

# batch_size = 32
# num_classes = 2
# epochs = 15

In [9]:
print(x_test.shape)
print(y_test.shape)

(17280, 100, 100, 3)
(17280,)


In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications.mobilenet_v2 import MobileNetV2  
from keras.applications.mobilenet import MobileNet

input_shape = (100, 100, 3)
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model.trainable=False

x = Flatten()(base_model.output)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 101, 101, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  del sys.path[0]


In [15]:
history = model.fit(x_train, y_train, validation_split=0.5, 
          epochs=20, batch_size=512, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)

Train on 12960 samples, validate on 12960 samples
Epoch 1/20
12960/12960 [==============================] - 40s 3ms/step - loss: 1.0369 - accuracy: 0.6658 - val_loss: 48.1578 - val_accuracy: 0.3617
Epoch 2/20
12960/12960 [==============================] - 28s 2ms/step - loss: 0.1996 - accuracy: 0.9217 - val_loss: 24.3691 - val_accuracy: 0.4544
Epoch 3/20
12960/12960 [==============================] - 28s 2ms/step - loss: 0.0930 - accuracy: 0.9650 - val_loss: 22.0848 - val_accuracy: 0.4515
Epoch 4/20
12960/12960 [==============================] - 28s 2ms/step - loss: 0.0554 - accuracy: 0.9788 - val_loss: 11.8485 - val_accuracy: 0.5352
Epoch 5/20
12960/12960 [==============================] - 28s 2ms/step - loss: 0.0422 - accuracy: 0.9840 - val_loss: 11.6921 - val_accuracy: 0.5575
Epoch 6/20
12960/12960 [==============================] - 28s 2ms/step - loss: 0.0285 - accuracy: 0.9896 - val_loss: 12.6861 - val_accuracy: 0.5785
Epoch 7/20
12960/12960 [==============================] - 28s 

In [13]:
model.save('save/MobileNet(50000).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [16]:
model.save('save/MobileNetV2(50000).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [17]:
# !unzip -uq "roi_1.zip" -d "dataset_pred/"

In [19]:
test_path = "dataset_pred/E01" # 기쁨 
test_path2 = "dataset_pred/E02" # 무표정 
test_path3 = "dataset_pred/E03" # 찡그림

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [20]:
# test set(기쁨)
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

In [22]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [24]:
# test set(찡그림)
read_dir1(test_path3)
X = np.array(X)
pred = model.predict(X)

In [21]:
# 예측(test set 기쁨)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n', "%.2f" % (co/2880.))


1012111111121022111211111211212122211111121221011102211211122121112111211212111111112122111111222121112121221211011121112211211112111221111112212111111111121211111110111111111211111101112112121112212211111211111111211122101211121111212122011212111111121221211112111121211111211112111110112122111212111121012111121121112111011211121121121211111111111111111111221111211111111111222121221112111212112111111111112211121121120111111121101111111111111111102111211121211111112122111112111111221110201111212121022111112111112112121122222212122211111111011121110211111121112211102112212211121121112121112211121111111111121021110111111121122111111111112111211111112221221111101211121111121211121121121111121111101111112111211112122111111111211112222122121211111111122111111211220111121110211101112211111121121111111112221111111111111122211202122111121221111221122121112021111121111211221212121221222101111211111111212111112111112210222111112111111111101211111121111121221121211211212211100021111210111112111111

In [23]:
# 예측(test set 무표정)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

1012111111121022111211111211212122211111121221011102211211122121112111211212111111112122111111222121112121221211011121112211211112111221111112212111111111121211111110111111111211111101112112121112212211111211111111211122101211121111212122011212111111121221211112111121211111211112111110112122111212111121012111121121112111011211121121121211111111111111111111221111211111111111222121221112111212112111111111112211121121120111111121101111111111111111102111211121211111112122111112111111221110201111212121022111112111112112121122222212122211111111011121110211111121112211102112212211121121112121112211121111111111121021110111111121122111111111112111211111112221221111101211121111121211121121121111121111101111112111211112122111111111211112222122121211111111122111111211220111121110211101112211111121121111111112221111111111111122211202122111121221111221122121112021111121111211221212121221222101111211111111212111112111112210222111112111111111101211111121111121221121211211212211100021111210111112111111

In [33]:
# 예측(test set 찡그림)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 2:
        co+=1

print('\n', "%.2f" % (co/2880.))

1012111111121022111211111211212122211111121221011102211211122121112111211212111111112122111111222121112121221211011121112211211112111221111112212111111111121211111110111111111211111101112112121112212211111211111111211122101211121111212122011212111111121221211112111121211111211112111110112122111212111121012111121121112111011211121121121211111111111111111111221111211111111111222121221112111212112111111111112211121121120111111121101111111111111111102111211121211111112122111112111111221110201111212121022111112111112112121122222212122211111111011121110211111121112211102112212211121121112121112211121111111111121021110111111121122111111111112111211111112221221111101211121111121211121121121111121111101111112111211112122111111111211112222122121211111111122111111211220111121110211101112211111121121111111112221111111111111122211202122111121221111221122121112021111121111211221212121221222101111211111111212111112111112210222111112111111111101211111121111121221121211211212211100021111210111112111111

In [0]:
# Total params : 3,256,515
# 28s 2ms/step
# acc : 기쁨(0.7194444444444444), 무표정(0.03125), 찡그림(0.24930555555555556)